# Cours LangChain TP1

Dans ce TP vous allez d'abord apprendre à créez des Prompt Templates et comprenez comment fournir des instructions au LLM.

preinstall et export de clé

In [ ]:
!pip install -U langchain-openai langchain langchainhub

In [2]:
import os
os.environ['OPENAI_API_KEY']="OPENAI_API_KEY" # À Modifier

Initialise le LLM

In [3]:
from langchain_openai import OpenAI

# initialize the models
llm = OpenAI(
    model_name="gpt-3.5-turbo-instruct",
)

### Exercice 1
Créez un simple `PromptTemplate` pour un ChatBot qui doit guider des passagers dans un centre commercial.
Rajoutez en contexte que le foot-locker se trouve au 3e étage du bâtiment.

In [5]:
from langchain import PromptTemplate

template = """Vous êtes un assistant de centre commercial qui doit guider les passager
            Context le foot locker se trouve au troisième étage du batiment

            Question: {query}

            Réponse:"""

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [6]:
#Call LLM avec le prompt
print(llm.invoke(
    prompt_template.format(
        query="Bonjour! Où se trouve le foot-locker?"
    )
))

 Bonjour! Le foot-locker se trouve au troisième étage du bâtiment. Suivez simplement les indications pour y arriver. Est-ce que vous avez besoin de plus d'aide pour trouver votre chemin?


### Exercice 2

Langchain vous donne accès à un grand panel de PromptTemplate à traver le [hub](https://smith.langchain.com/hub/search?q=french&organizationId=9a13fe78-6323-5784-993c-9714ceb93dd5)

le But de cette exercice est d'analyser les variables d'un promptTemplate pour voir comment le call avec notre LLM
Ici le avec template [conversation-title-generator](https://smith.langchain.com/hub/takatost/conversation-title-generator?organizationId=9a13fe78-6323-5784-993c-9714ceb93dd5&tab=0)

In [7]:
from langchain import hub
prompt = hub.pull("takatost/conversation-title-generator")

Quelles variables d'entrées prennent le template?

***Répondez ici***

En fonction de votre réponse faites un simple call de votre LLM avec le prompt template ci dessus

In [8]:
llm.invoke(
    "Bonjour ca va?"
)

'\n\nBonjour, oui ça va merci et vous ?'

### Exercice 3

Vous voulez utiliser un LLM pour essayer de créer des recettes à partir d'ingrédients que vous avez en reste chez vous.

Créez un Prompt template prenant deux variables. Un variable `{type_plat}`pour savoir quel type de plat vous voulez cuisiner et une `ingredients`contenant la liste d'ingrédients que vous voulez cuisiner.

Après avoir crée votre prompt template. Appelez votre LLM avec les entrées suivantes

In [9]:
plat="Dessert"

ingredients="de la semoule, de l'huile d'olive, du persil, des patates et du chocolat"

In [27]:
prompt_template= PromptTemplate.from_template("Donne-moi une recette de/d' {plat} avec les ingredients suivants:{ingredients}")
llm.invoke(prompt_template.format(plat=plat,ingredients=ingredients))

"\n\nRecette de Gâteau de semoule aux patates et au chocolat :\n\nIngrédients :\n- 200g de semoule fine\n- 2 cuillères à soupe d'huile d'olive\n- 1 bouquet de persil frais\n- 2 patates moyennes\n- 100g de chocolat noir\n- 50g de sucre en poudre\n- 1 sachet de levure chimique\n- 1 pincée de sel\n- 1 cuillère à soupe de cacao en poudre\n- 50g de beurre\n- 2 œufs\n- 1 cuillère à soupe de lait\n\nPréparation :\n\n1. Épluchez et coupez les patates en petits cubes. Faites-les cuire dans une casserole d'eau bouillante pendant 15 minutes. Égouttez-les et écrasez-les à la fourchette pour obtenir une purée.\n\n2. Dans un saladier, mélangez la semoule, le sucre, la levure, le sel et le cacao en poudre"

### Exercice 4

Few Shot Template

L'objectif de ce prompt FewShot est de fournir des exemples liés au cinéma et de guider le LLM dans la génération de réponses basées sur des requêtes liées au cinéma.

In [36]:
example = [
{"query" : "Qui a réalisé le film 'Inception' ?",
 "answer" : "Le réalisateur de 'Inception' est Christopher Nolan"
},{
"query" : "Quelle est l'intrigue du film 'The Shawshank Redemption' ?",
"answer": "L'intrigue du film 'The Shawshank Redemption' tourne autour d'un banquier condamné à la prison à vie de Shawshank pour le meurtre de sa femme et de l'amant de celle-ci."
},{
"query" : "Quel acteur a joué le rôle de 'Forrest Gump' dans le film 'Forrest Gump' ?",
"answer" : "Le personnage de 'Forrest Gump' a été joué par Tom Hanks."
},{
"query" : "Quel est le film qui a rapporté le plus d'argent de tous les temps ?",
"answer" : "Le film qui a rapporté le plus d'argent de tous les temps est 'Avatar'"
}
]

Créer un FewShotPromptTemplate : Utilisez la bibliothèque LangChain pour créer un objet FewShotPromptTemplate qui comprend les exemples liés au cinéma et un formateur pour les exemples de quelques plans.
Guidez le modèle : Demandez au modèle d'utiliser ces exemples pour comprendre les requêtes liées au cinéma et y répondre efficacement.

In [37]:
from langchain import FewShotPromptTemplate

#TO DO
# create a example template
example_template = """
Question: {query}
Réponse: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """Voici des extraits de conversations avec un assistant d'IA.
L'assistant est généralement sarcastique et plein d'esprit, produisant des réponses créatives et amusantes aux questions des utilisateurs. 
Voici quelques exemples: 
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=example,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [38]:
query = "Qui joue dans Titanic?"

print(llm.invoke(few_shot_prompt_template.format(query=query)))

Eh bien, il y a ce petit acteur nommé Leonardo DiCaprio et une certaine Kate Winslet. Peut-être les avez-vous déjà entendus parler ?


### Exercice 5
ChatModel et LLM

Reprenons le cas d'utilisation de l'exercice 1
Essayons de mettre ce contexte juste pour un ChatModel

In [10]:
from langchain_openai import ChatOpenAI

from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0.5)

Créez un `ChatPromptTemplate` d'un assistant IA d'un centre commercial qui aura son nom en paramètre et une question de l'utilisateurs

In [12]:
template = ChatPromptTemplate.from_template("Tu es un assistant IA qui s'appelle {name} et qui réponds aux question de l'utilisateurs: {user_input}")

messages = template.format_messages(
    name="Idriss",
    user_input="Où sommes-nous?"
)

In [13]:
chat.invoke(messages)

AIMessage(content="Nous sommes actuellement sur une plateforme de discussion en ligne. Comment puis-je vous aider aujourd'hui?")

# Exercice 6

Output Parser

À partir de la sortie du LLM ci-dessous, implémentez un `OutputParser` pour avoir la sortie du LLM en format JSON

In [19]:
from langchain_core.output_parsers import JsonOutputParser

prompt = hub.pull("ripemetrics/aspect-based-sentiment-analysis")
llm = OpenAI(model="gpt-3.5-turbo-instruct",temperature=0)

In [20]:
# Create runnable
runnable = prompt | llm #basic chain, on verra ca en detail au prochain TP

output = runnable.invoke({"user_comment":"Aujourd'hui j'ai eu une promotion grâce au nouveau produit que j'ai déployé!"})

In [24]:
# Implement the JSON Output Parser

#TO DO
parser= JsonOutputParser()
parser.invoke(output)

{'aspects': [{'text': 'promotion',
   'sentiment': {'score': 0.9, 'label': 'positive'}},
  {'text': 'nouveau produit',
   'sentiment': {'score': 0.9, 'label': 'positive'}},
  {'text': 'déployé', 'sentiment': {'score': 0.9, 'label': 'positive'}}]}